In [6]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [7]:
SCORE_DIR = "data/Scores"

In [8]:
box_scores = os.listdir(SCORE_DIR)

In [11]:
box_scores = [os.path.join(SCORE_DIR , f) for f in box_scores if f.endswith(".html")]

In [13]:
def parse_data(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup


In [159]:
def get_scores(soup):
    line_score = pd.read_html(str(soup), attrs = {"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0]= "TEAM"
    cols[-1] = "Final"
    line_score.columns = cols
    
    
    line_score = line_score[["TEAM", "Final"]]
    return line_score
                        

In [160]:
def parse_stats(soup,team, stat):
    df = pd.read_html(str(soup), attrs = {"id": f"box-{team}-game-{stat}"}, index_col = 0)[0]
    df = df.apply(pd.to_numeric, errors = "coerce")
    return df

In [161]:
def parse_season(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [162]:
# base_cols = None
# box_score = box_scores[0]

# # for box_score in box_scores:
# soup = parse_data(box_score)
# line_score = get_scores(soup)
# teams = list(line_score["TEAM"])

# summaries = []
# for team in teams:
#     basic = parse_stats(soup,team,"basic")
#     advanced = parse_stats(soup,team, "advanced")
    
#     totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
#     totals.index = totals.index.str.lower()
    
#     max_stats = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
#     max_stats.index = max_stats.index.str.lower() + "_max"
    
#     summ = pd.concat([totals,max_stats])
    
#     if base_cols is None:
#         base_cols = list (summ.index.drop_duplicates(keep = "first"))
#         base_cols = [b for b in base_cols if "bpm" not in b]
    
#     summ = summ[base_cols]
#     summaries.append(summ)
    
# summ = pd.concat(summaries, axis =1).T       


# game = pd.concat([summ, line_score], axis = 1)
# game ["home"] = [0,1]
# opponent_game = game.iloc[::-1]
# opponent_game.columns += "_opp"

# full_game = pd.concat([game, opponent_game], axis=1)

# full_game["season"] = parse_season(soup)

# full_game["date"] = os.path.basename(box_score)[0:8]
# full_game["date"] = pd.to_datetime(full_game["date"], format = "%Y%m%d")
# full_game["won"] = full_game["Final"] > full_game["Final_opp"]


In [174]:
games = []
base_cols = None

for box_score in box_scores:
    soup = parse_data(box_score)

    line_score = get_scores(soup)
    teams = list(line_score["TEAM"])

    summaries = []
    for team in teams:
        basic = parse_stats(soup, team, "basic")
        advanced = parse_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = parse_season(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["Final"] > full_game["Final_opp"]

    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 2038
200 / 2038
300 / 2038
400 / 2038
500 / 2038
600 / 2038
700 / 2038
800 / 2038
900 / 2038
1000 / 2038
1100 / 2038
1200 / 2038
1300 / 2038
1400 / 2038
1500 / 2038
1600 / 2038
1700 / 2038
1800 / 2038
1900 / 2038
2000 / 2038


In [175]:
games_data = pd.concat(games, ignore_index = True)

In [176]:
games_data

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,TEAM_opp,Final_opp,home_opp,season,date,won
0,265.0,265.0,46.0,100.0,0.460,15.0,39.0,0.385,25.0,32.0,...,20.7,39.9,160.0,125.0,NOP,138,1,2020,2020-01-16,False
1,265.0,265.0,51.0,93.0,0.548,11.0,26.0,0.423,25.0,35.0,...,25.0,39.5,197.0,132.0,UTA,132,0,2020,2020-01-16,True
2,240.0,240.0,48.0,91.0,0.527,20.0,41.0,0.488,13.0,21.0,...,50.0,33.5,147.0,141.0,TOR,105,1,2021,2021-03-03,True
3,240.0,240.0,34.0,77.0,0.442,12.0,36.0,0.333,25.0,30.0,...,25.2,31.2,197.0,115.0,DET,129,0,2021,2021-03-03,False
4,240.0,240.0,38.0,83.0,0.458,12.0,28.0,0.429,16.0,20.0,...,41.0,32.0,180.0,118.0,DAL,115,1,2021,2021-03-10,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4071,240.0,240.0,45.0,84.0,0.536,16.0,33.0,0.485,15.0,17.0,...,30.0,33.8,146.0,133.0,MIL,99,0,2023,2023-04-22,True
4072,240.0,240.0,34.0,82.0,0.415,9.0,33.0,0.273,10.0,16.0,...,26.8,33.4,164.0,99.0,DEN,103,1,2019,2019-02-11,False
4073,240.0,240.0,38.0,89.0,0.427,16.0,37.0,0.432,11.0,13.0,...,41.5,25.7,226.0,117.0,MIA,87,0,2019,2019-02-11,True
4074,240.0,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True


In [179]:
games_data.to_csv("nba_data_small.csv")